# Create Project CERES Deliverables for Tennessee farm and home science (agrtfn)

In [1]:
# importing and options
import time
from pathlib import Path
from shutil import copy

import pandas as pd
from ipywidgets import IntProgress, Label, VBox
from IPython.display import display

# == display 95% width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# set pandas option to display wider column info, use -1 to expand to largest necessary width
pd.set_option('max_colwidth', -1)

## Load data from CSV

In [2]:
# exported Google Docs spreadsheet as csv
meta_df = pd.read_csv('data/agrtfn_crl_deliverables.csv')
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app,crl_deliverable_directory
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921",NaN
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921",NaN
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925",NaN
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925",NaN
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925",NaN


In [3]:
# drop unfilled crl_deliverable_directory
meta_df = meta_df.drop(labels='crl_deliverable_directory', axis=1)
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921"
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921"
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925"
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925"
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925"


## Split date from title

In [4]:
# split by comma and space then get everything after first match
meta_df['title'].str.split(', ', n=1).str[1].head(5)

0    January 2, 1925
1    January 9, 1921
2    July 4, 1921   
3    July 11, 1921  
4    July 16, 1921  
Name: title, dtype: object

In [5]:
# add date_range to Dataframe by rsplitting on comma and taking the last field
meta_df['date'] = meta_df['title'].str.split(', ', n=1).str[1]
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app,date
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921","January 2, 1925"
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921","January 9, 1921"
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925","July 4, 1921"
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925","July 11, 1921"
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925","July 16, 1921"


## Convert date into yyyy, mm, dd

In [6]:
# split date range by spaces and get the first month
meta_df['date'].str.split().str[0].head(5)

0    January
1    January
2    July   
3    July   
4    July   
Name: date, dtype: object

In [7]:
# add the month to the DataFrame
meta_df['Month'] = meta_df['date'].str.split().str[0]
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app,date,Month
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921","January 2, 1925",January
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921","January 9, 1921",January
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925","July 4, 1921",July
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925","July 11, 1921",July
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925","July 16, 1921",July


In [8]:
# split yyyy from date range and add to Dataframe
meta_df['yyyy'] = meta_df['date'].str.split().str[-1]
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app,date,Month,yyyy
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921","January 2, 1925",January,1925
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921","January 9, 1921",January,1921
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925","July 4, 1921",July,1921
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925","July 11, 1921",July,1921
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925","July 16, 1921",July,1921


In [9]:
# convert month to mm, e.g. January -> 01
str(time.strptime(meta_df['Month'][0], '%B').tm_mon).zfill(2)

'01'

In [10]:
# create a list of what the mm column should be
# due to formatting issues getting time.strptime to work on a Pandas column
mm_list = []
for index, value in enumerate(meta_df['Month']):
    try:
        mm = str(time.strptime(value, '%B').tm_mon).zfill(2)
    except ValueError:
        print(index, value)
    mm_list.append(mm)
mm_list[:6]

['01', '01', '07', '07', '07', '07']

In [11]:
# add mm column to DataFrame using list
meta_df['mm'] = mm_list
meta_df.tail(5)

,adminDB,title,date_from_utk_metadata_app,date,Month,yyyy,mm
3445,0012_004266_003447,"Tennessee farm and home news, April 15, 1946",NaN,"April 15, 1946",April,1946,04
3446,0012_004266_003448,"Tennessee farm and home news, September 15, 1947",NaN,"September 15, 1947",September,1947,09
3447,0012_004266_003449,"Tennessee farm and home news, April 2, 1951",NaN,"April 2, 1951",April,1951,04
3448,0012_004266_003450,"Tennessee farm and home news, January 27, 1947",NaN,"January 27, 1947",January,1947,01
3449,0012_004266_003453,"Tennessee farm and home news, March 20, 1939",NaN,"March 20, 1939",March,1939,03


In [12]:
# add dd to DataFrame
meta_df['dd'] = meta_df['date'].str.split().str[-2].str.strip(',').str.zfill(2)
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app,date,Month,yyyy,mm,dd
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921","January 2, 1925",January,1925,01,02
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921","January 9, 1921",January,1921,01,09
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925","July 4, 1921",July,1921,07,04
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925","July 11, 1921",July,1921,07,11
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925","July 16, 1921",July,1921,07,16


## Map old to new directory

In [13]:
# explicitly define adminDB as old_directory_name
meta_df['old_directory_name'] = meta_df['adminDB']
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app,date,Month,yyyy,mm,dd,old_directory_name
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921","January 2, 1925",January,1925,01,02,0012_004266_000001
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921","January 9, 1921",January,1921,01,09,0012_004266_000002
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925","July 4, 1921",July,1921,07,04,0012_004266_000003
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925","July 11, 1921",July,1921,07,11,0012_004266_000004
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925","July 16, 1921",July,1921,07,16,0012_004266_000005


In [14]:
# add new_directory_name from yyyy, mm, and dd columns
meta_df['new_directory_name'] = meta_df['yyyy'].map(str) + '_' + meta_df['mm'].map(str) + '_' + meta_df['dd'].map(str)
meta_df.head(5)

,adminDB,title,date_from_utk_metadata_app,date,Month,yyyy,mm,dd,old_directory_name,new_directory_name
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921","January 2, 1925",January,1925,01,02,0012_004266_000001,1925_01_02
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921","January 9, 1921",January,1921,01,09,0012_004266_000002,1921_01_09
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925","July 4, 1921",July,1921,07,04,0012_004266_000003,1921_07_04
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925","July 11, 1921",July,1921,07,11,0012_004266_000004,1921_07_11
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925","July 16, 1921",July,1921,07,16,0012_004266_000005,1921_07_16


In [15]:
# verify that we have a matching number of old_ and new_directory_name(s)
meta_df.count()

adminDB                       3450
title                         3450
date_from_utk_metadata_app    1517
date                          3450
Month                         3450
yyyy                          3450
mm                            3450
dd                            3450
old_directory_name            3450
new_directory_name            3450
dtype: int64

In [16]:
dir_names = []
extra_dir_names = []
for new_directory_name in meta_df.new_directory_name:
    if new_directory_name not in dir_names:
        dir_names.append(new_directory_name)
    else:
        print(new_directory_name)
        extra_dir_names.append(new_directory_name)

In [17]:
for index, value in enumerate(meta_df.new_directory_name):
    if value in extra_dir_names:
        print('*****')
        print(value)
        print(meta_df.iloc[index])
        print('=====')

## Batch process directories

In [39]:
class Metadata:
    def __init__(self, directory_path):
        self.directory_path = Path(directory_path)
    
        
    def get_image_paths_list(self):
        
        image_paths_list = sorted(self.directory_path.glob('*.tif'))
        
        # delete all macOS index files that start with '.'
        deleted_dot_files = False
        for image_path in image_paths_list:
            if image_path.name.startswith('.'):
                image_path.unlink()  # delete
                deleted_dot_files = True
        if deleted_dot_files:  # re-create the list
            image_paths_list = sorted(self.directory_path.glob('*.tif'))
        
        self.image_paths_list = image_paths_list
        self.number_of_images = len(self.image_paths_list)
        
        return self.image_paths_list
    
    def process(self):
        
        # progress bar
        progress_label = Label('Images being processed')
        progress_bar = IntProgress(min=0, max=self.number_of_images)
        progress_widget = VBox([progress_label, progress_bar])
        display(progress_widget)
        
        # get output directory name
        output_directory_name = meta_df[meta_df["old_directory_name"].str.contains(directory_path.name)]['new_directory_name'].to_string(index=False)
        # strip spaces from output_directory_name
        self.output_directory_name = output_directory_name.strip()
        # set output directory path and make directory
        self.output_directory_path = batch_output_directory_path.joinpath(self.output_directory_name)
        self.output_directory_path.mkdir()
        
        # for each *.tif in self.directory_path
        for index, image_path in enumerate(self.image_paths_list, start=1):
            
            # update progress bar label
            progress_label.value = f'Processing image: {image_path.name} . . . {index}/{self.number_of_images}'
            
            # NO extension at the end for Tesseract processing
            pdf_output_path = self.output_directory_path.joinpath(f'{str(index).zfill(3)}')
            
            # OCR with Tesseract
            !tesseract {str(image_path)} {str(pdf_output_path)} pdf 2>/dev/null
            
            # set output path with extension at end
            pdf_output_path = self.output_directory_path.joinpath(f'{str(index).zfill(3)}.pdf')
            
            # update progress bar value
            progress_bar.value = index
            
        self.pdf_paths_list = list(self.output_directory_path.glob('*.pdf'))
        self.number_of_pdfs = len(self.pdf_paths_list)
        
        if self.number_of_pdfs != self.number_of_pdfs:
            print(f'# of PDFs != # of images to OCR')
            print(f'PDFs: {self.number_of_pdfs}')
            print(f'images: {self.number_of_images}')
            return None
        else:
            return self.pdf_paths_list
        
    def copy_pdfs(self):
        
        # progress bar
        progress_label = Label('Copying PDFs . . .')
        progress_bar = IntProgress(min=0, max=self.number_of_images)
        progress_widget = VBox([progress_label, progress_bar])
        display(progress_widget)
        
        # get output directory name
        output_directory_name = meta_df[meta_df["old_directory_name"].str.contains(directory_path.name)]['new_directory_name'].to_string(index=False)
        # strip spaces from output_directory_name
        self.output_directory_name = output_directory_name.strip()
        # set output directory path and make directory
        self.output_directory_path = batch_output_directory_path.joinpath(self.output_directory_name)
        self.output_directory_path.mkdir()
            
        self.pdf_paths_list = list(self.directory_path.glob('*.pdf'))
        # delete all macOS index files that start with '.'
        deleted_dot_files = False
        for pdf_path in self.pdf_paths_list:
            if pdf_path.name.startswith('.'):
                pdf_path.unlink()  # delete
                deleted_dot_files = True
        if deleted_dot_files:  # re-create the list
            self.pdf_paths_list = sorted(self.directory_path.glob('*.pdf'))
            
        self.number_of_pdfs = len(self.pdf_paths_list)
        
        if self.number_of_images != self.number_of_pdfs:
            print(f'input_dir: {self.directory_path}')
            print(f'# of PDFs != # of images to OCR')
            print(f'PDFs: {self.number_of_pdfs}')
            print(f'images: {self.number_of_images}')
            return None
        else:
            # for each *.pdf in self.directory_path
            for index, pdf_path in enumerate(self.pdf_paths_list, start=1):

                # update progress bar label
                progress_label.value = f'Copying PDF: {pdf_path.name} . . . {index}/{self.number_of_pdfs}'

                # set output path with extension at end
                pdf_output_path = self.output_directory_path.joinpath(pdf_path.name)

                # copy file
                copy(pdf_path, pdf_output_path)

                # update progress bar value
                progress_bar.value = index
                
        self.pdf_paths_list = list(self.output_directory_path.glob('*.pdf'))
        # delete all macOS index files that start with '.'
        deleted_dot_files = False
        for pdf_path in self.pdf_paths_list:
            if pdf_path.name.startswith('.'):
                pdf_path.unlink()  # delete
                deleted_dot_files = True
        if deleted_dot_files:  # re-create the list
            self.pdf_paths_list = sorted(self.output_directory_path.glob('*.pdf'))
        self.number_of_pdfs = len(self.pdf_paths_list)
        if self.number_of_images != self.number_of_pdfs:
            print(f'input_dir: {self.directory_path}')
            print(f'output_dir: {self.output_directory_path}')
            print(f'# of PDFs != # of images to OCR')
            print(f'PDFs: {self.number_of_pdfs}')
            print(f'images: {self.number_of_images}')
            return None

In [40]:
# Set batch_input and batch_output directories

# set project identifier and root directory path
project_identifier = 'agrtfn'
root_directory_path = Path('/Volumes/fluffy/ProjectCeres/00_for_CRL/')

# set batch_input and batch_output directory paths from root and project identifier
batch_input_directory_path = root_directory_path.joinpath(project_identifier)
batch_output_directory_path = root_directory_path.joinpath(f'{project_identifier}_batch_output')
batch_output_directory_path.mkdir()

# create batch_input directory paths list
batch_input_directory_paths_list = sorted([x for x in batch_input_directory_path.iterdir() if x.is_dir()])
number_of_input_dirs = len(batch_input_directory_paths_list)

print(f'batch_input directory: {batch_input_directory_path}')
print(f'\t{number_of_input_dirs} directories to batch process\n')
print(f'batch_output directory: {batch_output_directory_path}')

batch_input directory: /Volumes/fluffy/ProjectCeres/00_for_CRL/agrtfn
	3450 directories to batch process

batch_output directory: /Volumes/fluffy/ProjectCeres/00_for_CRL/agrtfn_batch_output


In [41]:
# start batch process

# progress bar
progress_label = Label('Directories being processed')
progress_bar = IntProgress(min=0, max=number_of_input_dirs)
progress_widget = VBox([progress_label, progress_bar])
display(progress_widget)

for index, directory_path in enumerate(batch_input_directory_paths_list, start=1):
    
    # update progress bar label
    progress_label.value = f'Processing directory: {directory_path.name} . . . {index}/{number_of_input_dirs}'
    
    # load directory path as class & process
    issue = Metadata(directory_path)
    issue.get_image_paths_list()
    issue.copy_pdfs()
    
    #update progress bar value
    progress_bar.value = index

print(f'Processed {index} directories')

Processed 3450 directories


In [ ]:
df

In [ ]:
output_df = df_tn_farm_and_home_science['new_directory_name']

In [44]:
meta_df.to_clipboard(index=False, header=False, excel=True, sep='\t')

In [ ]:
from pathlib import Path

In [43]:
meta_df

,adminDB,title,date_from_utk_metadata_app,date,Month,yyyy,mm,dd,old_directory_name,new_directory_name
0,0012_004266_000001,"Tennessee farm news, January 2, 1925","January 2, 1921","January 2, 1925",January,1925,01,02,0012_004266_000001,1925_01_02
1,0012_004266_000002,"Tennessee farm news, January 9, 1921","January 9, 1921","January 9, 1921",January,1921,01,09,0012_004266_000002,1921_01_09
2,0012_004266_000003,"Tennessee farm news, July 4, 1921","July 13, 1925","July 4, 1921",July,1921,07,04,0012_004266_000003,1921_07_04
3,0012_004266_000004,"Tennessee farm news, July 11, 1921","July 13, 1925","July 11, 1921",July,1921,07,11,0012_004266_000004,1921_07_11
4,0012_004266_000005,"Tennessee farm news, July 16, 1921","July 13, 1925","July 16, 1921",July,1921,07,16,0012_004266_000005,1921_07_16
5,0012_004266_000006,"Tennessee farm news, July 25, 1921","July 25, 1921","July 25, 1921",July,1921,07,25,0012_004266_000006,1921_07_25
6,0012_004266_000007,"Tennessee farm news, August 1, 1921","August 1, 1921","August 1, 1921",August,1921,08,01,0012_004266_000007,1921_08_01
7,0012_004266_000008,"Tennessee farm news, August 8, 1921","August 8, 1921","August 8, 1921",August,1921,08,08,0012_004266_000008,1921_08_08
8,0012_004266_000009,"Tennessee farm news, August 15, 1921","August 15, 1921","August 15, 1921",August,1921,08,15,0012_004266_000009,1921_08_15
9,0012_004266_000010,"Tennessee farm news, August 22, 1921","August 22, 1921","August 22, 1921",August,1921,08,22,0012_004266_000010,1921_08_22
